# Throughtput Benchmarking  Seldon-Core on GCP Kubernetes

The notebook will provide a benchmarking of seldon-core for maximum throughput test. We will run a stub model and test using REST and gRPC predictions. This will provide a maximum theoretical throughtput for model deployment in the given infrastructure scenario:
  
   * 1 replica of the model running on n1-standard-16 GCP node
   
For a real model the throughput would be less. Future benchmarks will test realistic models scenarios.


## Create Cluster

Create a cluster of 4 nodes of machine type n1-standard-16. You can use GKE console or `gcloud` command line.

## Cordon off loadtest nodes

In [ ]:
!kubectl get nodes

We cordon off first 3 nodes so seldon-core and the model will not be deployed on the 1 remaining node.

In [ ]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

Label the nodes so they can be used by locust.

In [ ]:
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}') role=locust
!kubectl label nodes $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}') role=locust

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

## Create Stub Deployment

In [ ]:
!pygmentize resources/loadtest_simple_model.json

In [ ]:
!kubectl apply -f resources/loadtest_simple_model.json

Wait for deployment to be running.

In [ ]:
!kubectl rollout status deployment.apps/loadtest-default-9eecb7d

## Run benchmark

Uncorden the first 3 nodes so they can be used to schedule locust

In [ ]:
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl uncordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

## gRPC
Start locust load test for gRPC

In [ ]:
!helm install loadtest ../helm-charts/seldon-core-loadtesting \
    --set locust.host=loadtest-seldon-core-loadtest:5001 \
    --set locust.script=predict_grpc_locust.py \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=256 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=64 

To download stats use 

```bash
if [ "$#" -ne 2 ]; then
    echo "Illegal number of parameters: <experiment> <rest|grpc>"
fi

EXPERIMENT=$1
TYPE=$2

MASTER=`kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}'`

kubectl cp ${MASTER}:stats_distribution.csv ${EXPERIMENT}_${TYPE}_stats_distribution.csv
kubectl cp ${MASTER}:stats_requests.csv ${EXPERIMENT}_${TYPE}_stats_requests.csv
```

You can get live stats by viewing the logs of the locust master

In [ ]:
!kubectl logs $(kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}') --tail=10

In [ ]:
!helm delete loadtest

## REST 
Run REST benchmark

In [ ]:
!helm install loadtest ../helm-charts/seldon-core-loadtesting \
    --set locust.host=http://loadtest-seldon-core-loadtest:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=256 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=64

Get stats as per gRPC and/or monitor

In [ ]:
!kubectl logs $(kubectl get pod -l name=locust-master-1 -o jsonpath='{.items[0].metadata.name}') --tail=10

In [ ]:
!helm delete loadtest

In [ ]:
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[0].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[1].metadata.name}')
!kubectl cordon $(kubectl get nodes -o jsonpath='{.items[2].metadata.name}')

## Tear Down

In [ ]:
!kubectl delete -f resources/loadtest_simple_model.json

In [ ]:
!helm delete seldon-core